# Simple S-Exp Parser

Grammar:
- P -> id | E 
- L -> P | P L
- E ->  (L)

LL(1):
- P -> id | E 
- L -> PL'
- L' -> L | null
- E ->  (L)


In [1]:
class parser:
    '''
    # S-Exp Parser
        Grammar:
            P -> id | E 
            L -> P | P L
            E -> (L)

        LL(1):
            P  -> id | E 
            L  -> PL'
            L' -> L | null
            E  -> (L)
    '''

    def lexer(s):
        s = s.replace('\n', ' ').replace('(', ' ( ').replace(')', ' ) ')
        l = s.split()
        ret = []
        for c in l:
            if c == '(':
                ret.append(('(', None))
            elif c == ')':
                ret.append((')', None))
            else:
                ret.append(('id', c))
        return ret

    def __init__(self, s):
        self.tokens = list(reversed(parser.lexer(s)))

    def run(self):
        return self.E()

    def next(self):
        self.tokens.pop()

    def peek(self):
        return self.tokens[-1]

    def P(self):
        token = self.peek()
        if token[0] == 'id':
            # print("P id={}".format(token[1]))
            self.next()
            return token[1]
        else:
            ret = self.E()
            return ret

    def L(self):
        token = self.peek()
        ret = [self.P()]
        ret += self.Lp()
        return ret

    def Lp(self):
        token = self.peek()
        if token[0] == 'id' or token[0] == '(':
            ret = self.L()
            return ret
        else:
            return []

    def E(self):
        token = self.peek()
        assert token[0] == '('
        self.next()
        ret = self.L()
        token = self.peek()
        assert token[0] == ')'
        self.next()
        return ret


In [2]:
test_parser = parser("(define z (λ (f) (λ (x) x)))")
test_parser.run()

['define', 'z', ['λ', ['f'], ['λ', ['x'], 'x']]]

In [3]:
prog01 = '''
((define z
       (λ (f)
         (λ (x)
           x)))
     (define s
       (λ (n)
         (λ (f)
           (λ (x)
             (f ((n f) x))))))
     (s (s z))))
'''
test_parser = parser(prog01)
test_parser.run()

[['define', 'z', ['λ', ['f'], ['λ', ['x'], 'x']]],
 ['define',
  's',
  ['λ', ['n'], ['λ', ['f'], ['λ', ['x'], ['f', [['n', 'f'], 'x']]]]]],
 ['s', ['s', 'z']]]